In [1]:
# The Sequential Model

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# when to use a sequential model: A Sequential Model is appropriate for a plain
# stack of layers where each layer has exactly one input tensor and one output tensor

# Define Sequential model with 3 layers
model = tf.keras.Sequential([
    tf.keras.layers.Dense(2, activation="relu", name="layer1"),
    tf.keras.layers.Dense(3, activation="relu", name="layer2"),
    tf.keras.layers.Dense(4, name="layer3")
])

# call model on a test input
x = tf.ones((3,3))
y = model(x)


In [3]:
'''
A sequential model is not appropriate when:
    1. your model has multiple inputs or multiple outputs
    2. Any of your layers has multiple inputs or multiple outputs
    3. You need to do layer sharing
    4. You want non-linear topology (e.g. a residual connection, a multi-branch model)

Note: There is also a corresponding pop() method to remove layers:
a sequential model behaves very much like a list of layers.

'''
print(model.layers)

[<tensorflow.python.keras.layers.core.Dense object at 0x7fc61b7325c0>, <tensorflow.python.keras.layers.core.Dense object at 0x7fc61a86ed68>, <tensorflow.python.keras.layers.core.Dense object at 0x7fc61a86ea58>]


In [5]:
layer = layers.Dense(3)
layer.weights
# it create its weights the first time called on an input, since the shape of the
# weights depends on the shape of the inputs:

# call layer on a test input
x = tf.ones((1,4))
y = layer(x)
layer.weights

'''
Models built with a predefined input shape like this always have weights (even before
seeing any data) and always have a defined output shape.
'''

[<tf.Variable 'dense_1/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[ 0.41573596,  0.59593153, -0.87636095],
        [ 0.7407646 , -0.09603465,  0.45464432],
        [ 0.79758275, -0.31994504,  0.17598164],
        [-0.16361725, -0.49288005, -0.8967176 ]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [6]:
'''
A common debugging workflow : add() + summary()

when building a new Sequential architecture, it is useful to incrementally stack layers
with add() and frequently print model summaries. For instance, this enables you to monitor
how a stack of Conv2D and MaxPooling2D layers is downsampling image feature maps.

'''
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3))) # 250*250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPool2D(3))

In [7]:
# Transfer learning with a Sequential model
'''
Transfer learning consists of freezing the bottom layers in a model and only training the top layers.

'''
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(10),
])

# presumably you would want to first load pre-trained weights
model.load_weights(...)

# freeze all layers except the last one.
for layer in model.layers[:-1]:
    layer.trainable = False

# recompile and train the model ( this will only updates the weights of the last layer)
model.compile(...)
model.fit(...)


AttributeError: 'ellipsis' object has no attribute 'endswith'